In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
import json
import gc
from pathlib import Path

import cv2
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.colors import ListedColormap
import webcolors
from PIL import Image, ImageDraw

from minimal.layout import InputLayout
from minimal.imaging import draw_plan
from minimal.gen import run_model

%matplotlib inline

In [3]:
from minimal import sample_graphs

from pregen.premade import premade_graphs
from pregen import signature as sg

In [11]:
# print(len(premade_graphs))
# for g in premade_graphs:
#     path = "plibrary/" + sg.graph_folder_name(g)
#     if not Path(path).is_dir():
#         print(path)

48


In [18]:
prev = {}
for i, g in enumerate(premade_graphs):
    s = sg.graph_folder_name(g)
    if s in prev:
        print(prev[s], s)
        print(i)
    prev[s] = i

31 557dbca12dfbd63a0794
33
